Definizione di tutti i metodi usati. **AVVIARE PRIMA DI QUALSIASI ALTRO CODICE**

In [201]:
import json
import re
import time

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderTimedOut, GeocoderQuotaExceeded

from sklearn.neighbors import BallTree
import numpy as np


with open('CL_ITTER107.json', 'r') as file:
    data = json.load(file)

# Estrai la lista di codici dal file JSON
codes = data['data']['codelists'][0]['codes']

# Funzione per trovare l'id in base al nome del comune
def find_id_comune_by_name(name):
    for code in codes:
        if code['name']['it'] == name and re.match("^\d{6}$", code['id']):
            return code['id']
    return None

# Funzione per trovare l'id in base al nome della regione
def find_id_regione_by_name(name):
    for code in codes:
        if code['name']['it'] == name and re.match("^IT.*$", code['id']):
            return code['id']
    return None

# Funzione per ottenere gli abitanti di un comume
def extract_abitanti_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Funzione per ottenere i problemi in una regione
def extract_problemi_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0:0:0:0:0:0:0:0:0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Funzione per ottenere le denunce in una regione
def extract_denunce_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Funzione per ottenere la spesa media in una regione
def extract_spesamedia_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0:0:0:0:0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Funzione per ottenere la superficie di un comune
def extract_superficie_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Funzione per ottenere le aree a rischio idrogeologico in km2
def extract_rischi_idro_observation(response_json):
    try:
        observationHIGH = response_json['data']['dataSets'][0]['series']['0:0:0']['observations']['0'][0]
        observationLOW = response_json['data']['dataSets'][0]['series']['0:0:1']['observations']['0'][0]
        observationMED = response_json['data']['dataSets'][0]['series']['0:0:2']['observations']['0'][0]
        observations= [observationLOW, observationMED, observationHIGH]
        return observations
    except KeyError:
        return None

# Funzione per ottenere location da un comune
def trova_location(comune):
    geolocator = Nominatim(user_agent="FindYourPlace")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=5)

    try:
      structuredQuery = {
        "postalcode" : '',
        "state" : '',
        "country" : 'Italia',
        "street" : '',
        "city" : {comune}
      }

      location = geocode(structuredQuery)
      if location is None:
        location = geocode(comune)
    except (GeocoderTimedOut, GeocoderQuotaExceeded, TimeoutError) as e:
      if (TimeoutError, GeocoderQuotaExceeded):
            print(e)
      else:
            print(f'Location not found: {e}')
            return None

    if location is None:
        print(f'Location for {comune} not found')
    return location

# Funzione per ottenere le coordinate di un comune
def trova_coordinate(comune):
    loc= trova_location(comune)

    if loc:
        latitudine, longitudine = loc.latitude, loc.longitude
        return latitudine, longitudine
    else:
        return None

# Funzione per ottenere il bounding box di un comune
def get_bounding_box(comune, bbox_type):
    valid = {"have", "overpass"}
    if bbox_type not in valid:
      raise ValueError("results: type must be one of %r." % valid)

    loc = trova_location(comune)

    if loc:
        bounding_box = loc.raw.get('boundingbox', None)
        if bounding_box:
            if bbox_type=="have":
              return bounding_box[2], bounding_box[0], bounding_box[3], bounding_box[1]
            elif bbox_type=="overpass":
              return bounding_box[0], bounding_box[2], bounding_box[1], bounding_box[3]

    return None

# Funzione per ottenere le coordinate più vicine in un dataframe
def trova_coordinate_vicine(latitudine, longitudine, dataframe):
    bt = BallTree(np.deg2rad(dataframe[['LAT', 'LON']].values), metric='haversine')
    distanze, indici = bt.query(np.deg2rad(np.c_[latitudine, longitudine]))

    nearest = dataframe.iloc[indici[0]]

    return nearest[['LAT', 'LON', 'Zona Sismica']]



>Trova popolazione in base al nome del comune

In [ ]:
import requests
import re

# Trova id tramite nome comune
comune_to_find = 'Salerno'
id_comune_found = find_id_comune_by_name(comune_to_find)

if id_comune_found:
    print(f"L'id corrispondente al nome '{comune_to_find}' è: {id_comune_found}")
else:
    print(f"Nome '{comune_to_find}' non trovato.")

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/22_289/.TOTAL.{id_comune_found}.9.99.?startPeriod=2023&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_abitanti_observation(response.json())

    if observation is not None:
        print(f"L'observation della popolazione estratto è: {observation}")
    else:
        print("Observation della popolazione non trovato nella risposta JSON.")
else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


L'id corrispondente al nome 'Salerno' è: 065116
L'observation estratto è: 127485


Trova presenza di problemi nella zona di residenza delle famiglie, come inquinamento, rumori o criminalità.<br>
Trova numero segnalazioni di denunce e arresti.

In [ ]:
import requests
import re
from enum import Enum

# Trova id tramite nome regione oppure Nord-Centro-Sud
regione_to_find = 'Campania'
id_regione_found = find_id_regione_by_name(regione_to_find)

if id_regione_found:
    print(f"L'id corrispondente al nome '{regione_to_find}' è: {id_regione_found}")
else:
    print(f"Nome '{regione_to_find}' non trovato.")

class Problema(Enum):
    INQUINAMENTO = 1
    RUMORI = 2
    CRIMINALITA = 3

problema_scelto= Problema.INQUINAMENTO

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/33_291/A.{id_regione_found}...{problema_scelto.value}..........?startPeriod=2022&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_problemi_observation(response.json())

    if observation is not None:
        print(f"L'observation di {problema_scelto.name} estratto è: {observation}")
    else:
        print(f"Observation di {problema_scelto.name} non trovato nella risposta JSON.")
else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


#### Segnalazioni relative a persone denunciate e arresti

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/73_342/A.{id_regione_found}.REPN.TOT.?startPeriod=2022&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_denunce_observation(response.json())

    if observation is not None:
        print(f"L'observation delle denunce estratto è: {observation}")
    else:
        print(f"Observation delle denunce non trovato nella risposta JSON.")
else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


L'id corrispondente al nome 'Campania' è: ITF3
L'observation di INQUINAMENTO estratto è: 22.3
L'observation delle denunce estratto è: 71256


Trova spesa media per consumi

In [ ]:
import requests
import re
from enum import Enum

# Trova id tramite nome regione oppure Nord-Centro-Sud
regione_to_find = 'Campania'
id_regione_found = find_id_regione_by_name(regione_to_find)

if id_regione_found:
    print(f"L'id corrispondente al nome '{regione_to_find}' è: {id_regione_found}")
else:
    print(f"Nome '{regione_to_find}' non trovato.")

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/31_739/A.HH.99.SPESA_MEDIA..ALL..{id_regione_found}.ALL.ALL.TOT?startPeriod=2021&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_spesamedia_observation(response.json())

    if observation is not None:
        print(f"L'observation della spesa media estratto è: {observation}")

        costo_vita= None
        if observation < 2100:
          costo_vita= "BASSO"
        elif observation > 2700:
          costo_vita= "ALTO"
        else:
          costo_vita= "MEDIO"
        print(f"Il Costo della Vita corrisponde a: {costo_vita}")
    else:
        print(f"Observation della spesa media non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


L'id corrispondente al nome 'Campania' è: ITF3
L'observation della spesa media estratto è: 2034.65
Il Costo della Vita corrisponde a: BASSO


Trova aree a pericolosità idraulica in km2

In [ ]:
import requests
import re
from enum import Enum

# Trova id tramite nome comune
comune_to_find = 'Venezia'
id_comune_found = find_id_comune_by_name(comune_to_find)

if id_comune_found:
    print(f"L'id corrispondente al nome '{comune_to_find}' è: {id_comune_found}")
else:
    print(f"Nome '{comune_to_find}' non trovato.")

####Superficie di un comune
# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/729_1050/A.{id_comune_found}.TOTAREA?startPeriod=2023&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

superficie= None
# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_superficie_observation(response.json())

    if observation is not None:
        print(f"L'observation della superficie estratto è: {observation}")
        superficie= observation/100
    else:
        print(f"Observation della superficie non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


####Rischio idrogeologico in km2
# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/729_995/A.{id_comune_found}.LOWFLHZ+MEDFLHZ+HIGHFLHZ?startPeriod=2017&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

rischio_idro= None
# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observations = extract_rischi_idro_observation(response.json())

    if observations is not None:
        print(f"L'observationLOW estratto è: {observations[0]}")
        print(f"L'observationMED estratto è: {observations[1]}")
        print(f"L'observationHIGH estratto è: {observations[2]}")

        rischio_idro= (observations[0] + observations[1] + observations[2])/superficie*100
        print(f"Il rischio idrogeologico è: {rischio_idro}")

    else:
        print(f"Observation non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


L'id corrispondente al nome 'Venezia' è: 027042
L'observation della superficie estratto è: 41589.27
L'observationLOW estratto è: 128.4
L'observationMED estratto è: 24.97
L'observationHIGH estratto è: 12.11
Il rischio idrogeologico è: 39.78910906587204


Estrazione Zone Sismiche.

In [ ]:
import pandas as pd

# Caricamento dei dati
sismi = pd.read_csv('italiaMedianaSismi.csv', sep=';')

def convert_percent(x):
    return float(x.replace(",", "."))

sismi["ag"] = sismi["ag"].apply(convert_percent)
sismi["LON"] = sismi["LON"].apply(convert_percent)
sismi["LAT"] = sismi["LAT"].apply(convert_percent)

min_value = sismi["ag"].min()
max_value = sismi["ag"].max()
sismi["normalized_ag"] = 4 * (sismi["ag"] - min_value) / (max_value - min_value)

def calcola_rischio(mediana):
    if mediana >= 3.0:
        return 1
    elif 2.0 <= mediana < 3.0:
        return 2
    elif 1.0 <= mediana < 2.0:
        return 3
    else:
        return 4

#Maggiore il numero, più il rischio
sismi['Zona Sismica']= sismi['normalized_ag'].apply(calcola_rischio)

print(sismi[["LON", "LAT", "Zona Sismica"]])
sismi[["LON", "LAT", "Zona Sismica"]].to_csv("Zone Sismiche.csv", index=False)

            LON      LAT  Zona Sismica
0       12.0899  47.0710             4
1       12.1193  47.0713             4
2       12.1487  47.0716             4
3       12.1781  47.0719             4
4       12.2075  47.0723             4
...         ...      ...           ...
104560  12.6943  35.3374             4
104561  12.7187  35.3376             4
104562  12.7432  35.3378             4
104563  12.7676  35.3380             4
104564  12.7679  35.3179             4

[104565 rows x 3 columns]


Trova Zona Sismica di un comune

In [34]:
import pandas as pd

comune_da_cercare = "Napoli"
coordinate = trova_coordinate(comune_da_cercare)

if coordinate:
    print(f"Le coordinate di {comune_da_cercare} sono: {coordinate}")
else:
    print(f"Coordinate non trovate per {comune_da_cercare}")

if coordinate:
    zone_sismiche = pd.read_csv('Zone Sismiche.csv')

    # Trova le coordinate più vicine al punto di riferimento
    near_coords = trova_coordinate_vicine(coordinate[0], coordinate[1], zone_sismiche)

    print(f"Le coordinate più vicine a {comune_da_cercare} sono:")
    print(near_coords)
else:
    print(f"Coordinate non trovate per {comune_da_cercare}")

Le coordinate di Napoli sono: (40.8358846, 14.2487679)
Le coordinate più vicine a Napoli sono:
           LAT      LON  Zona Sismica
60116  40.8455  14.2452             3


Trova Bounding Box di un comune e ricerca numero negozi, ristoranti e scuole<br>
Purtroppo il limite di pois trovabili è 100, per cui mi limito a cercare in un raggio di X metri solo determinate categorie.

In [210]:
comune= 'Castel San Giorgio'
bounding_box = get_bounding_box(comune,"have")
coords= trova_coordinate(comune)

if bounding_box:
    print(f"Bounding box di {comune}: {bounding_box}")
else:
    print(f"Bounding box non trovato per {comune}")


API_KEY= "oY1k3tXVkAI8O68lu62eXTEkuOc7TQb6Pwn2S_ZCXKo"

### Trova Negozi
api_url = (f'https://geocode.search.hereapi.com/v1/browse?'
f'at={coords[0]},{coords[1]}'
#f'&in=bbox:{bounding_box[0]},{bounding_box[1]},{bounding_box[2]},{bounding_box[3]}'
f'&in=circle:{coords[0]},{coords[1]};r=1000'
#https://www.here.com/docs/bundle/places-search-api-developer-guide/page/topics/place_categories/category-600-shopping.html
f'&categories=600-6000,600-6100,600-6200,600-6600-0000,600-6700-0000'
f'&limit=100&apiKey={API_KEY}')

# Effettua la richiesta all'API
response = requests.get(api_url)

num= None
# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    num = len(response.json()["items"])

    if num is not None:
        print(f"Numero negozi estratto è: {num}")
    else:
        print(f"Items non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


### Trova Ristoranti
api_url = (f'https://geocode.search.hereapi.com/v1/browse?'
f'at={coords[0]},{coords[1]}'
#f'&in=bbox:{bounding_box[0]},{bounding_box[1]},{bounding_box[2]},{bounding_box[3]}'
f'&in=circle:{coords[0]},{coords[1]};r=200'
#https://www.here.com/docs/bundle/places-search-api-developer-guide/page/topics/place_categories/category-600-shopping.html
f'&categories=100-1000-0000'
f'&limit=100&apiKey={API_KEY}')

# Effettua la richiesta all'API
response = requests.get(api_url)

num= None
# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    num = len(response.json()["items"])

    if num is not None:
        print(f"Numero ristoranti estratto è: {num}")
    else:
        print(f"Items non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")

### Trova Scuole
api_url = (f'https://geocode.search.hereapi.com/v1/browse?'
f'at={coords[0]},{coords[1]}'
#f'&in=bbox:{bounding_box[0]},{bounding_box[1]},{bounding_box[2]},{bounding_box[3]}'
f'&in=circle:{coords[0]},{coords[1]};r=1000'
#https://www.here.com/docs/bundle/places-search-api-developer-guide/page/topics/place_categories/category-600-shopping.html
f'&categories=800-8200-0000,800-8200-0173,800-8200-0174'
f'&limit=100&apiKey={API_KEY}')

# Effettua la richiesta all'API
response = requests.get(api_url)

num= None
# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    num = len(response.json()["items"])

    if num is not None:
        print(f"Numero scuole estratto è: {num}")
    else:
        print(f"Items non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")

Bounding box di Castel San Giorgio: ('14.6525694', '40.7648780', '14.7182108', '40.7980006')
Numero negozi estratto è: 8
Numero ristoranti estratto è: 1
Numero scuole estratto è: 1


In questo codice invece provo ad usare overpass, ma in caso di comuni con pochi dati, questo risulterà inaffidabile.<br>
Infatti spesso il codice soprastante anche usando solo un certo raggio e certe categorie, riesce a trovare più negozi in un certo paese.

In [212]:
import requests

def trova_numero_negozi(bbox_query):
    overpass_endpoint = "https://overpass-api.de/api/interpreter"

    # Definisci la query Overpass
    query = f"""
    [out:json];
    (
        node["shop"]({bbox_query});
        way["shop"]({bbox_query});
        relation["shop"]({bbox_query});
    );
    out count;
    """

    # Parametri della richiesta
    params = {
        'data': query
    }

    # Effettua la richiesta
    response = requests.get(overpass_endpoint, params=params)

    # Estrai il numero di negozi dal risultato
    result = response.json()
    numero_negozi = result['elements'][0]['tags']['total']

    return numero_negozi

#specifica bbox
comune= 'Castel San Giorgio'
bbox = get_bounding_box(comune,"overpass")

bbox_query = f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}"

# Trova il numero di negozi nelle coordinate specificate
numero_negozi_trovati = trova_numero_negozi(bbox_query)

# Stampa il risultato
print(f"Numero di negozi nel bbox specificato: {numero_negozi_trovati}")

Numero di negozi nelle coordinate specificate: 4


Creazione dataset IdQ province con dati assegnati

In [47]:
import pandas as pd

province= pd.read_csv('IdQ_Province.csv')
print(province)

province[['Latitudine', 'Longitudine']] = province['Provincia'].apply(lambda provincia: pd.Series(trova_coordinate(provincia)))

print(province)
province[['Provincia', 'Latitudine', 'Longitudine', 'IdQ']].to_csv("IdQ_Prov_Coords.csv", index=False)





                 Provincia        IdQ
0                Agrigento  32.705827
1    Barletta-Andria-Trani  43.064425
2                   Varese  61.216379
3                  Venezia  60.845987
4     Verbano-Cusio-Ossola  56.126538
..                     ...        ...
102                Trapani  34.712002
103                 Trento  73.595235
104                Treviso  63.369359
105                Trieste  69.587881
106                  Udine  71.950673

[107 rows x 2 columns]
                 Provincia        IdQ  Latitudine  Longitudine
0                Agrigento  32.705827   37.312299    13.574650
1    Barletta-Andria-Trani  43.064425   41.180172    16.146641
2                   Varese  61.216379   45.817549     8.826353
3                  Venezia  60.845987   45.437191    12.334590
4     Verbano-Cusio-Ossola  56.126538   46.115576     8.383597
..                     ...        ...         ...          ...
102                Trapani  34.712002   38.017432    12.515992
103             